# Tutorial: VisionKG - A Data-Centric Way to Train your own Obejct Detection Models

# 1. QuickView of VisionKG

## 1.1 One-Click to meet VisionKG

In [ ]:
%load_ext autoreload
%autoreload 2

from google.colab import output
# install our vision api
!python -m pip install git+https://github.com/cqels/vision.git --force
output.clear()

## 1.2 Query a Dataset as YOU need via VisionKG

In [ ]:
# import SemkgAPI
from vision_utils import semkg_api, data
from skimage import io
import matplotlib.pyplot as plt
from torch_model_zoo import utils

query_string='''# Give me 100 images contain Car and Truck with its annotations.
PREFIX cv:<http://vision.semkg.org/onto/v0.1/>
PREFIX schema:<http://schema.org/>
PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd:<http://www.w3.org/2001/XMLSchema#>
SELECT ?datasetName (STR(?_imageName) AS ?imageName) ?imageUrl (xsd:integer(?_imageWidth) AS ?imageWidth) (xsd:integer(?_imageHeight) AS ?imageHeight) ?labelName ?bbCentreX ?bbCentreY ?bbWidth ?bbHeight
WHERE {
  {
    SELECT ?image
    WHERE{
      ?image cv:hasAnnotation ?annotation1.
      ?annotation1 a cv:ObjectDetectionAnnotation.
      ?annotation1 cv:hasLabel ?label1.
      ?label1 cv:label "car".

      ?image cv:hasAnnotation ?annotation2.
      ?annotation2 a cv:ObjectDetectionAnnotation.
      ?annotation2 cv:hasLabel ?label2.
      ?label2 cv:label "truck".

      ?image schema:isPartOf / schema:name ?datasetName .
      FILTER regex(?datasetName, "coco2017", "i")
    }
    GROUP BY ?image
    LIMIT 100
  }
  ?image schema:isPartOf / schema:name ?datasetName .
  ?image schema:name ?_imageName.
  OPTIONAL{?image schema:contentUrl ?imageUrl}.
  ?image cv:hasAnnotation ?annotation.
  ?image cv:imgWidth ?_imageWidth.
  ?image cv:imgHeight ?_imageHeight.
  ?annotation cv:hasLabel/cv:label ?labelName.
  ?annotation cv:hasBox ?bbox.
  ?bbox cv:boxHeight ?bbHeight.
  ?bbox cv:boxWidth ?bbWidth.
  ?bbox cv:centerX ?bbCentreX.
  ?bbox cv:centerY ?bbCentreY.
}
'''

#Query and return result
result=semkg_api.query(query_string)
result = utils.visionkg2cocoDet(result)

# 2. Object Detection in Practice starting from VisionKG

## 2.1 Prepare and set parameters for training and evaluation

In [ ]:
from torch_model_zoo import utils
from os.path import join as opj

filter_cat_nms = ['car', 'truck']
path_to_anno_mixedDatasets = opj('data/mixedDatasets/','test_query_api_image.json')
params = utils.prepare_for_training(path_to_anno_mixedDatasets, result, filter_cat_nms)

##2.2 Dataset Playground

In [ ]:
if params['NUM_CAT'] > 4:
  cat_nms = params['CAT_NMS'][0:4]
else:
  cat_nms = params['CAT_NMS']

utils.show_annotation(path_to_anno_mixedDatasets, cat_nms, show_num=6)
utils.show_cat_distribution(path_to_anno_mixedDatasets, cat_nms)

## 2.3 Perform Training & Evaluation on pytorch build-in models

In [ ]:
from torch_model_zoo.train_eval import train_eval_pipeline
params['IMGS_PATH'] = 'data/image_dataset/coco'
train_eval_pipeline(params)